In [1]:
email="vaibhavjain055@gmail.com"
website="www.vaibhav.com"

In [2]:
import requests
headers = { "User-Agent": f"{website} {email}"}
form = "https://www.sec.gov/Archives/edgar/data/320193/000032019322000063/wf-form4_165248105838188.xml"
res = requests.get(form)

In [3]:
res = requests.get(form, headers=headers)
res.raise_for_status()
res.content[0:50]

b'<?xml version="1.0"?>\n<ownershipDocument>\n\n    <sc'

In [4]:
from lxml import etree
doc = etree.fromstring(res.content)
owner = doc.xpath("/ownershipDocument/reportingOwner/reportingOwnerId/rptOwnerName")[0].text
security = doc.xpath("//securityTitle/value")[0].text
date = doc.xpath("//transactionDate/value")[0].text
amount = doc.xpath("//transactionAmounts/transactionShares/value")[0].text
disposal = doc.xpath("//transactionAmounts/transactionAcquiredDisposedCode/value")[0].text
owner, security, date, amount, disposal

('BELL JAMES A', 'Common Stock', '2022-05-06', '1276', 'D')

In [5]:
symbol_to_cik = requests.get("https://www.sec.gov/files/company_tickers.json").json()
for i in range(5):
    print(i, symbol_to_cik[f"{i}"])



0 {'cik_str': 320193, 'ticker': 'AAPL', 'title': 'Apple Inc.'}
1 {'cik_str': 789019, 'ticker': 'MSFT', 'title': 'MICROSOFT CORP'}
2 {'cik_str': 1318605, 'ticker': 'TSLA', 'title': 'Tesla, Inc.'}
3 {'cik_str': 1067983, 'ticker': 'BRK-B', 'title': 'BERKSHIRE HATHAWAY INC'}
4 {'cik_str': 731766, 'ticker': 'UNH', 'title': 'UNITEDHEALTH GROUP INC'}


In [6]:
cik_lookup = dict([(val['ticker'], val['cik_str']) for key, val in symbol_to_cik.items()])
cik = cik_lookup['INTC']
cik

50863

In [7]:
edgar_filings = requests.get(f"https://data.sec.gov/submissions/CIK{cik:0>10}.json", headers=headers).json()
edgar_filings.keys()

dict_keys(['cik', 'entityType', 'sic', 'sicDescription', 'insiderTransactionForOwnerExists', 'insiderTransactionForIssuerExists', 'name', 'tickers', 'exchanges', 'ein', 'description', 'website', 'investorWebsite', 'category', 'fiscalYearEnd', 'stateOfIncorporation', 'stateOfIncorporationDescription', 'addresses', 'phone', 'flags', 'formerNames', 'filings'])

In [8]:
import pandas as pd
recents = pd.DataFrame(edgar_filings['filings']['recent'])
recents.head()

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription
0,0000950123-22-008991,2022-08-15,2022-06-30,2022-08-15T06:46:41.000Z,34,13F-HR,028-04527,221162237,,7935,0,0,xslForm13F_X01/primary_doc.xml,
1,0001193125-22-218541,2022-08-11,2022-08-08,2022-08-11T16:11:07.000Z,34,8-K,000-06217,221155840,"5.02,7.01,9.01",194551,1,1,d359017d8k.htm,8-K
2,0001193125-22-213699,2022-08-05,2022-08-02,2022-08-05T16:54:34.000Z,34,8-K,000-06217,221141549,"8.01,9.01",1066541,1,1,d390659d8k.htm,8-K
3,0001127602-22-020426,2022-08-03,2022-08-01,2022-08-03T17:49:56.000Z,,4,,,,8219,0,0,xslF345X03/form4.xml,PRIMARY DOCUMENT
4,0001127602-22-020425,2022-08-03,2022-08-01,2022-08-03T17:48:19.000Z,,4,,,,8196,0,0,xslF345X03/form4.xml,PRIMARY DOCUMENT


In [9]:
recents['reportDate'] = pd.to_datetime(recents['reportDate'])
recents['filingDate'] = pd.to_datetime(recents['filingDate'])

In [10]:
insider_q1 = recents[(recents['form'] == "4") &
        (recents['filingDate'] >= '2022-01-01') &
        (recents['filingDate'] <= '2022-03-31')]
insider_q1.shape

(19, 14)

In [11]:
def make_url(cik, row):
    accessionNumber = row['accessionNumber'].replace("-", "")
    return f"https://www.sec.gov/Archives/edgar/data/{cik}/{accessionNumber}/{row['accessionNumber']}.txt"
make_url(cik, insider_q1.iloc[0])

'https://www.sec.gov/Archives/edgar/data/50863/000112760222010351/0001127602-22-010351.txt'

In [12]:
from bs4 import BeautifulSoup
req = requests.get(make_url(cik, insider_q1.iloc[0]), headers=headers)
soup = BeautifulSoup(req.content, 'html.parser')
req.content[0:300]

b'<SEC-DOCUMENT>0001127602-22-010351.txt : 20220324\n<SEC-HEADER>0001127602-22-010351.hdr.sgml : 20220324\n<ACCEPTANCE-DATETIME>20220324185112\nACCESSION NUMBER:\t\t0001127602-22-010351\nCONFORMED SUBMISSION TYPE:\t4\nPUBLIC DOCUMENT COUNT:\t\t1\nCONFORMED PERIOD OF REPORT:\t20220322\nFILED AS OF DATE:\t\t20220324\nD'

In [13]:
str(soup.find_all('xml')[0])[0:300]


'<xml>\n<?xml version="1.0"?>\n<ownershipdocument>\n<schemaversion>X0306</schemaversion>\n<documenttype>4</documenttype>\n<periodofreport>2022-03-22</periodofreport>\n<issuer>\n<issuercik>0000050863</issuercik>\n<issuername>INTEL CORP</issuername>\n<issuertradingsymbol>INTC</issuertradingsymbol>\n</issuer>\n<re'

In [14]:
import re
def get_document(cik, row):
    url = make_url(cik, row)
    res = requests.get(url, headers=headers)
    res.raise_for_status()
    soup = BeautifulSoup(res.content, 'html.parser')
    # use a case insensitive search for the root node of the XML document
    docs = soup.find_all(re.compile("ownershipDocument", re.IGNORECASE))
    if len(docs) > 0:
        doc = etree.fromstring(str(docs[0]))
        if docs[0].name == "ownershipDocument":
            owner = doc.xpath("/ownershipDocument/reportingOwner/reportingOwnerId/rptOwnerName")[0].text
            security = doc.xpath("//securityTitle/value")[0].text
            date = doc.xpath("//transactionDate/value")[0].text
            amount = doc.xpath("//transactionAmounts/transactionShares/value")[0].text
            disposal = doc.xpath("//transactionAmounts/transactionAcquiredDisposedCode/value")[0].text
        elif docs[0].name == "ownershipdocument":
            owner = doc.xpath("/ownershipdocument/reportingowner/reportingownerid/rptownername")[0].text
            security = doc.xpath("//securitytitle/value")[0].text
            date = doc.xpath("//transactiondate/value")[0].text
            amount = doc.xpath("//transactionamounts/transactionshares/value")[0].text
            disposal = doc.xpath("//transactionamounts/transactionacquireddisposedcode/value")[0].text
            pass
        else:
            raise ValueError(f"Don't know how to process {docs[0].name}")
        return (owner, security, date, amount, disposal)

In [15]:
get_document(cik, insider_q1.iloc[0])

('GELSINGER PATRICK P', 'Common Stock', '2022-03-22', '13064', 'A')

In [16]:
results = []
for i, row in insider_q1.iterrows():
    try:
        results.append(get_document(cik, row))
    except ValueError as ve:
        print(f"{ve} on row {i}")
pd.DataFrame(results, columns=['name', 'type', 'date', 'quantity', 'acquired_disposed' ])


,name,type,date,quantity,acquired_disposed
0,GELSINGER PATRICK P,Common Stock,2022-03-22,13064,A
1,"Intel Americas, Inc.",Class A Common Stock,2022-03-01,169742322,A
2,ISHRAK OMAR,Common Stock,2022-02-22,11025,A
3,GELSINGER PATRICK P,Common Stock,2022-02-22,5600,A
4,GELSINGER PATRICK P,Common Stock,2022-02-15,35135,A
5,Zinsner David,Performance-Based Stock Units (PSUs),2022-01-31,78031,A
6,Rodgers Steven Ralph,Common Stock,2022-01-31,56030,A
7,Rivera Sandra L,Common Stock,2022-01-31,41147,A
8,GELSINGER PATRICK P,Performance-Based Stock Units (PSUs),2022-01-31,325366,A
9,WEISLER DION J,Restricted Stock Units,2022-01-31,2152,A
